In [3]:
import datetime
import numpy as np
import os
import pandas as pd
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import logging

# Set a random seed for reproducibility
np.random.seed(1337)

# **************************************************************************************************************************************************
# Global Variables
# **************************************************************************************************************************************************

# Location of the test image
imagePath = '/Users/nurajymnaukanova/Desktop/dataset thyroid/test/Benign/2_0.jpg'

# Saved filename of model weights
weights_filename = '/Users/nurajymnaukanova/Downloads/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Saved filename of result CSV
results_filename = '/Users/nurajymnaukanova/Desktop/results.csv'

# Dimensions of the architecture
img_width, img_height = 224, 224

# Directories of dataset
test_data_dir = '/Users/nurajymnaukanova/Desktop/dataset thyroid/test'

# Number of channels for the architecture
channels = 3

# Total number of classes
total_classes = 2  # Binary classification

# Learning rate
learning_rate = 1e-4

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# **************************************************************************************************************************************************
# Model Setup
# **************************************************************************************************************************************************

# Load pre-trained ResNet50 model with custom output layer
model = applications.ResNet50(include_top=False, weights=None, input_shape=(img_width, img_height, channels))

# Load the pre-trained weights manually from the local file
model.load_weights(weights_filename, by_name=True)

# Flatten and custom Dense layer for binary classification
flat = Flatten()(model.output)
output = Dense(total_classes, activation="sigmoid")(flat)  # Sigmoid for binary classification

# Final model with custom layers
loaded_model = Model(inputs=model.inputs, outputs=output)

# Compile model with updated learning rate argument
loaded_model.compile(loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
                     optimizer=optimizers.SGD(learning_rate=learning_rate), 
                     metrics=['accuracy'])

# **************************************************************************************************************************************************
# Functions
# **************************************************************************************************************************************************

def test_image(path, output=True):
    """
    Function to test a single image and return its prediction.
    """
    try:
        # Load and process image
        test_image = image.load_img(path, target_size=(224, 224))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

        # Predict the result
        result = loaded_model.predict(test_image)

        # Format confidence values
        confidence = ["{:.10f}".format(r) for r in result[0]]

        if output:
            logger.info(f"Confidence: {confidence}")

        # Determine thyroid type based on prediction
        if result[0][0] > result[0][1]:  # Threshold logic for binary classification
            if output: 
                logger.info("Thyroid Type: Benign")
            return "Benign", confidence
        elif result[0][1] > result[0][0]:  
            if output: 
                logger.info("Thyroid Type: Malignant")
            return "Malignant", confidence
        else:
            if output: 
                logger.info("Thyroid Type: Unknown. Unable to classify this image.")
            return "Unknown", confidence
    except Exception as e:
        logger.error(f"Error processing image {path}: {e}")
        return "Error", ["N/A"]

# **************************************************************************************************************************************************
# Predict one image
# **************************************************************************************************************************************************

test_image(imagePath, output=True)

# **************************************************************************************************************************************************
# Predict all test images and save the results
# **************************************************************************************************************************************************

images = []
predictions = []
confidences = []

logger.info("Predicting all test images...")
start = datetime.datetime.now()

# Loop through all images in the test directory
count = 0
for (dirpath, dirnames, filenames) in os.walk(test_data_dir):
    for file in filenames:
        # Predict each image and store the result
        result, confidence = test_image(os.path.join(dirpath, file), output=False)
        images.append(file)
        predictions.append(result)
        confidences.append(confidence)
        count += 1

logger.info(f"Writing {count} results to CSV...")

# Save results to CSV file
results = pd.DataFrame({
    "Filename": images,
    "Prediction": predictions,
    "Confidence": confidences
})
results.to_csv(results_filename, index=False)

# Display elapsed time for the entire process
end = datetime.datetime.now()
elapsed = end - start
logger.info(f"Prediction completed after {elapsed}.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step


INFO:__main__:Confidence: ['0.1654153615', '0.1076671928']
INFO:__main__:Thyroid Type: Benign
INFO:__main__:Predicting all test images...
ERROR:__main__:Error processing image /Users/nurajymnaukanova/Desktop/dataset thyroid/test/.DS_Store: cannot identify image file <_io.BytesIO object at 0x358480a90>


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━

INFO:__main__:Writing 360 results to CSV...
INFO:__main__:Prediction completed after 0:00:24.219472.
